In [11]:
import splitwise
from splitwise import Splitwise
import sqlite3
from typing import List, Tuple, Dict, Optional
import dateutil.parser
import os
from httplib2 import ServerNotFoundError

In [12]:
def main():
    try:
        settings = get_settings()
    except FileNotFoundError:
        print("File settings.txt is not found.")
        return 

    consumer_key = settings.get("consumer_key", None)
    consumer_secret = settings.get("consumer_secret", None)
    
    # Check if there are application credentials in the settings file. Otherwise, ask user to add them
    if consumer_key is None or consumer_secret is None:
        print("Add conusmer_key and consumer_secret values to the settings file.")
        return
    sObj = Splitwise(consumer_key, consumer_secret)
    
    oauth_token = settings.get("oauth_token", None)
    oauth_token_secret = settings.get("oauth_token_secret", None)
    
    # Check if there are Splitwise credentials in the settings file. Otherwise, redirect 
    # user to authorization page and ask him to enter his credentials
    if oauth_token is None or oauth_token_secret is None:
        token = get_access_token_from_user(sObj)
        settings.update(token)
        save_settings_to_file(settings)
    else:
        token = {"oauth_token": oauth_token, "oauth_token_secret": oauth_token_secret}
    sObj.setAccessToken(token)
    
    try:
        curr_user = sObj.getCurrentUser()
    except ServerNotFoundError:
        print("Cannot connect to the server. Check your connection")
        return
    except Exception as e:
        print(type(e))
        print("Invalid credentials. Try again.")
        return
        
    db_name = "{}.sqlite".format(curr_user.getId())
        
    db_exists = os.path.exists(db_name)
    
    with sqlite3.connect(db_name) as connection:
        if not db_exists:
            create_tables(connection)
        print("Saved transactions")
        print_table_rows("Transactions", connection)
        update_users(sObj, connection)
        update_groups(sObj.getGroups(), connection)
        update_categories(sObj.getCategories(), connection)
        update_subcategories(sObj.getCategories(), connection)
        update_transactions(sObj, connection)
        connection.commit()
        print("Transactions after update")
        print_table_rows("Transactions", connection)

    return connection

In [13]:
def print_table_rows(table_name, connection):
    """Prints all rows from the specified table"""
    cursor = connection.execute("SELECT * FROM {}".format(table_name))
    row_names = [description[0] for description in cursor.description]
    print(*row_names)
    for row in cursor:
        print(row)

In [14]:
def get_settings() -> Dict[str, str]:
    """Returns a dict with settings from settings.txt file"""
    res = {}
    
    with open("settings.txt", "r") as settings_file:
        for line in settings_file:
            # split by "=" and remove spaces
            param, value = map(str.strip, line.split("="))
            res[param] = value
    return res

In [15]:
def get_access_token_from_user(sObj: Splitwise) -> Dict[str, str]:
    """Asks a user to go to authorize url and enter his verifier"""
    url, secret = sObj.getAuthorizeURL()
    
    # oauth_token is a query param. The next line extracts it from the url.
    oauth_token = url[url.find("=") + 1:]
    print("Go to the following url: {}".format(url))
    print("Then, press \"Authorize\" button. After that click on \"Click to show out of band authentication information\" text and enter your verifier")
    
    # User should copy oauth_verifier from the web page
    oauth_verifier = input("oauth_verifier: ")
    access_token = sObj.getAccessToken(oauth_token, secret, oauth_verifier)
    return access_token

In [16]:
def save_settings_to_file(settings: Dict[str, str]) -> None:
    """Saves settings so settings.txt file"""
    with open("settings.txt", "w+") as file:
        for key, value in settings.items():
            print("{}={}".format(key, value), file=file)

In [17]:
def create_tables(connection: sqlite3.Connection) -> None:
    """Creates all necessary tables"""
    connection.execute("""
    CREATE TABLE Users(
        userID INTEGER PRIMARY KEY,
        name TEXT 
    );
    """)
    
    connection.execute("""
    CREATE TABLE Groups(
        groupID INTEGER PRIMARY KEY,
        groupName TEXT
    );
    """)
    
    connection.execute("""
    CREATE TABLE Categories(
        categoryID INTEGER PRIMARY KEY,
        category TEXT
    )
    """)
    
    connection.execute("""
    CREATE TABLE Subcategories(
        subcategoryID INTEGER PRIMARY KEY,
        category TEXT
    )
    """)
    
    connection.execute("""
    CREATE TABLE Transactions(
        transactionID INTEGER PRIMARY KEY,
        date TEXT,
        groupID INTEGER,
        subcategoryID INTEGER,
        description TEXT,
        currency TEXT,
        repeatInterval TEXT,
        updated TEXT
    )
    """)
    
    connection.execute("""
    CREATE TABLE TransactionItems(
        itemID INTEGER PRIMARY KEY AUTOINCREMENT,
        transactionID INTEGER,
        userID INTEGER,
        amount REAL,
        baseAmount REAL
    )
    """)
    connection.commit()

In [18]:
def update_users(sObj: Splitwise, connection: sqlite3.Connection) -> None:
    """Updates Users table"""
    curr_user = sObj.getCurrentUser()
    new_users = sObj.getFriends()
    new_users.append(curr_user)
    
    user_tuples = []
    
    # Convert all users to tuples to insert into table.
    for user in new_users:
        user_id = user.getId()
        user_name = user.getFirstName()
        last_name = user.getLastName()
        if last_name is not None:
            user_name += " " + last_name
        user_tuples.append((user_id, user_name))
    
    connection.executemany("INSERT OR REPLACE INTO Users VALUES (?, ?)", user_tuples)
    
def update_groups(groups: List[splitwise.group.Group], connection: sqlite3.Connection) -> None:
    """Updates Groups table"""
    group_tuples = [(group.getId(), group.getName()) for group in groups]
    connection.executemany("INSERT OR REPLACE INTO Groups VALUES (?, ?)", group_tuples)

def update_categories(categories: List[splitwise.category.Category], connection: sqlite3.Connection) -> None:
    """Updates Categories table"""
    categories_tuples = [(category.getId(), category.getName()) for category in categories]
    connection.executemany("INSERT OR REPLACE INTO Categories VALUES (?, ?)", categories_tuples)
    
def update_subcategories(categories: List[splitwise.category.Category], connection: sqlite3.Connection) -> None:
    """Updates Subcategories table"""
    subcategory_tuples = [
        (subcategory.getId(), subcategory.getName())
        for category in categories
        for subcategory in category.getSubcategories()
    ]
    connection.executemany("INSERT OR REPLACE INTO Subcategories VALUES (?, ?)", subcategory_tuples)

def update_transactions(sObj: Splitwise, connection: sqlite3.Connection) -> None:
    """Updates transactions table"""
    all_expenses = sObj.getExpenses()
    expense_update_times = get_update_times_from_db(connection)
    
    # Expenses that are not in the DB.
    new_expenses = [expense for expense in all_expenses if expense.getId() not in expense_update_times and expense.getDeletedAt() is None]
    
    # Expenses that are already in the DB, but that are not deleted
    existing_expenses = [expense for expense in all_expenses if expense.getId() in expense_update_times and expense.getDeletedAt() is None]
    
    # Expenses that need to be deleted
    expenses_to_delete = [expense for expense in all_expenses if expense.getId() in expense_update_times and expense.getDeletedAt() is not None]
    insert_new_transactions(new_expenses, connection)
    update_existing_transactions(expense_update_times, existing_expenses, connection)
    delete_transactions(expenses_to_delete, connection)

In [19]:
def get_update_times_from_db(connection: sqlite3.Connection) -> Dict[int, str]:
    """Returns a dict for every Transaction from DB where key is transaction's ID and value is its update time"""
    cursor = connection.execute("SELECT transactionId, updated from Transactions")
    return {transaction_id: updated for transaction_id, updated in cursor}

def insert_new_transactions(expenses: List[splitwise.expense.Expense], connection: sqlite3.Connection) -> None:
    """Inserts new transactions to the table"""
    connection.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?, ?, ?, ?)", expenses_to_transaction_tuples(expenses))
    
    for expense in expenses:
        insert_transaction_items_from_expense(expense, connection)

def update_existing_transactions(expense_update_times: Dict[int, str], existing_expenses: List[splitwise.expense.Expense], connection: sqlite3.Connection) -> None:
    """Updates transactions from the list if their update times are not equal to fetched expenses' times"""
    expenses_to_update = []
    
    for expense in existing_expenses:
        if expense_update_times[expense.getId()] == _iso_date_to_updated_format(expense.getUpdatedAt()):
            continue
        
        # Delete and replace all TransactionItems for every updated transaction
        connection.execute("DELETE FROM TransactionItems WHERE transactionId = ?", (expense.getId(),))
        insert_transaction_items_from_expense(expense, connection)
        expenses_to_update.append(expense)
    
    connection.executemany("REPLACE INTO Transactions VALUES (?, ?, ?, ?, ?, ?, ?, ?)", expenses_to_transaction_tuples(expenses_to_update))

def delete_transactions(expenses_to_delete: List[splitwise.expense.Expense], connection: sqlite3.Connection) -> None:
    """Deletes expenses from a passed list"""
    for expense in expenses_to_delete:
        expense_id = expense.getId()
        connection.execute("DELETE FROM TransactionItems WHERE transactionId = ?", (expense_id,))
        connection.execute("DELETE FROM Transactions WHERE transactionId = ?", (expense_id,))
    
ExpenseTuple = Tuple[int, str, int, int, str, str, str, str]

def expenses_to_transaction_tuples(expenses: List[splitwise.expense.Expense]) -> List[ExpenseTuple]:
    """Converts splitwise Expense objects to tuples to insert into DB"""
    res = []
    
    for expense in expenses:
        expense_id = expense.getId()
        expense_date = _iso_date_to_dd_mm_yyyy(expense.getDate())
        group_id = expense.getGroupId()
        subcategory_id = expense.getCategory().getId()
        description = expense.getDescription()
        currency = expense.getCurrencyCode()
        repeat_interval = expense.getRepeatInterval()
        updated = _iso_date_to_updated_format(expense.getUpdatedAt())
        res.append((expense_id, expense_date, group_id, subcategory_id, description, currency, repeat_interval, updated))
    return res

def _iso_date_to_dd_mm_yyyy(iso_date: str) -> str:
    dt = dateutil.parser.parse(iso_date)
    return dt.strftime("%d.%m.%Y")

def _iso_date_to_updated_format(iso_date: str) -> str:
    dt = dateutil.parser.parse(iso_date)
    return dt.strftime("%Y.%m.%d %H:%M")

def insert_transaction_items_from_expense(expense: splitwise.expense.Expense, connection: sqlite3.Connection) -> None:
    """Inserts transaction items from an expense"""
    connection.executemany("INSERT INTO TransactionItems ( transactionId, userId, amount, baseAmount) values(?, ?, ?, ?)", get_transaction_item_tuples_from_expense(expense))

TransactionItemTuple = Tuple[int, int, float, Optional[float]]

def get_transaction_item_tuples_from_expense(expense: splitwise.expense.Expense) -> List[TransactionItemTuple]:
    """"""
    res = []
    for expense_user in expense.getUsers():
        transaction_id = expense.getId()
        user_id = expense_user.getId()
        amount = float(expense_user.getOwedShare())
        base_amount = None
        res.append((transaction_id, user_id, amount, base_amount))
    return res

In [20]:
if __name__ == "__main__":
    main()

Saved transactions
transactionID date groupID subcategoryID description currency repeatInterval updated
Transactions after update
transactionID date groupID subcategoryID description currency repeatInterval updated
